In [ ]:
from __future__ import division
from __future__ import print_function

import sys
import argparse
import cv2
import editdistance
from DataLoader import DataLoader, Batch
from Model import Model, DecoderType
from SamplePreprocessor import preprocess
import os
import numpy as np
from PIL import Image
from WordSegmentation import wordSegmentation, prepareImg

In [ ]:
# Doing line segmentation
linewise_strings=[]
# import image
def getsegmented(path):
    image = cv2.imread(path)
    # cv2.imshow('orig',image)
    # cv2.waitKey(0)

    # grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray',gray)
    cv2.waitKey(0)

    # binary
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('second',thresh)
    cv2.waitKey(0)

    # dilation
    kernel = np.ones((5, 100), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    # cv2.imshow('dilated',img_dilation)
    cv2.waitKey(0)

    # find contours
    ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # sort contours
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    line_imgs = []
    line_path = '../data2/'
    img2=image
    for i, ctr in enumerate(sorted_ctrs):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr)

        # Getting ROI
        roi = image[y:y + h , x:x + w]
        line_imgs.append(roi)
        filename = str(i) + '.png'
        cv2.imwrite(line_path + filename, roi)
        # show ROI
        # cv2.imshow('segment no:'+str(i),roi)
        cv2.rectangle(img2, (x, y), (x + w, y + h), (90, 0, 255), 2)
        cv2.waitKey(0)

    cv2.imshow('marked areas', img2)
    cv2.imwrite('../segmented_fullimg/'+path[-8:],img2)
    cv2.waitKey(0)
    return line_imgs